# COG

## Initialise COG

### Load packages

In [1]:
%load_ext autoreload

import os
import certifi

# we need to set gdal and rasterio defaults. this will need thought.
os.environ['GDAL_DATA']  = r'C:\Program Files\ArcGIS\Pro\Resources\pedata\gdaldata' # in order for rasterio gdal affine and crs to work
os.environ.setdefault("CURL_CA_BUNDLE", certifi.where()) # in order for xr open_rasterioor rasterion.open to work

import sys
import html
import requests
import gdal
import rasterio
#import geopandas as gpd
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt

from datetime import datetime
from lxml import etree

sys.path.append('../../../shared')
import satfetcher

ModuleNotFoundError: No module named 'rasterio'

### Set up a dask cluster

In [50]:
# initialise the cluster
#create_local_dask_cluster()

## Get study area polygon

### Load study area geometry as geojson

In [51]:
# read study area and get bounds as a list
#gdf = gpd.read_file('../../data/cog/yandisa.geojson')
#gdf_bounds = gdf.bounds.values[0].tolist()

gdf_bounds = [118.92837524414061, -22.816061209792938, 
              119.16526794433592, -22.68118293381927]

## Set STAC Search parameters

In [52]:
# get satellite collection on dea. todo get from user in arcgis, sentinel 2
collections = [
    'ga_ls5t_ard_3', 
    'ga_ls7e_ard_3',
    'ga_ls8c_ard_3'
]

# set required bands
bands = [
    'nbart_blue', 
    'nbart_green', 
    'nbart_red', 
    'nbart_nir',
    'nbart_swir_1',
    'nbart_swir_2',
    'oa_fmask'
]

# get satellite collection date range, convert to stac. todo get from user in arcgis
start_dt, end_dt = '1990-01-01', '1995-12-31'

# bring it all together for a query
query = {
    'collections': collections,
    'datetime': '{0}/{1}'.format(start_dt, end_dt),
    'bbox': gdf_bounds,
    'query': {'eo:cloud_cover': {'lt': 5}}, #this doesnt work
    'limit': 1000
}

## Fetch DEA Public Data via STAC Search

In [53]:
# set stac endpoint
search_endpoint = 'https://explorer.sandbox.dea.ga.gov.au/stac/search'

# send and get request for stac json using 
stac_response = requests.post(search_endpoint, json=query)

# check for response empty errors, convert to json if so
if stac_response.ok:
    stac_response = stac_response.json()
    num_items = len(stac_response.get('features'))
    print('Found {0} satellite scenes in total.'.format(num_items))
else:
    raise ValueError('Could not connect to DEA STAC SEARCH endpoint.')

Found 146 satellite scenes in total.


## Iterate STAC response and remove cloud cover

In [54]:
# set max cloud cover (0 - 100)
max_cloud = 25

# get num of all stac scenes
num_all_items = len(stac_response.get('features'))

feat_list = []
for feat in stac_response.get('features'):
    if max_cloud > float(feat.get('properties').get('eo:cloud_cover')):
        feat_list.append(feat)
        
# count cloud less scenes and compare
if feat_list:
    num_clean_items = len(feat_list)
    print('Removed {0} satellite scenes due to clouds.'.format(num_all_items - num_clean_items))
    print('Total of {0} satellite scenes remaining.'.format(num_clean_items))

Removed 23 satellite scenes due to clouds.
Total of 123 satellite scenes remaining.


## Build VRTs for each band of each scene in STAC response

In [55]:
# get dict of band names and associated vrt lists
band_vrt_dict = satfetcher.get_dea_landsat_vrt_dict(feat_list)

Getting landsat vrts for each relevant bands.
Building landsat vrt list for band: nbart_blue.
Building landsat vrt list for band: nbart_green.
Building landsat vrt list for band: nbart_red.
Building landsat vrt list for band: nbart_nir.
Building landsat vrt list for band: nbart_swir_1.
Building landsat vrt list for band: nbart_swir_2.
Building landsat vrt list for band: oa_fmask.
Got 7 landsat vrt band lists successfully.


## Combine VRTs into file for each band set

In [56]:
# loop each band and combine vrts into one per band
vrt_file_dict = satfetcher.combine_vrts_per_band(band_vrt_dict)

Combining VRTs into single VRTs per band.
Combining VRTs into temp. file for band: nbart_blue.
<osgeo.gdal.Dataset; proxy of <Swig Object of type 'GDALDatasetShadow *' at 0x000001E815D5BD80> >
Combining VRTs into temp. file for band: nbart_green.
<osgeo.gdal.Dataset; proxy of <Swig Object of type 'GDALDatasetShadow *' at 0x000001E815D5BD80> >
Combining VRTs into temp. file for band: nbart_red.
<osgeo.gdal.Dataset; proxy of <Swig Object of type 'GDALDatasetShadow *' at 0x000001E815D5BD80> >
Combining VRTs into temp. file for band: nbart_nir.
<osgeo.gdal.Dataset; proxy of <Swig Object of type 'GDALDatasetShadow *' at 0x000001E815D5BD80> >
Combining VRTs into temp. file for band: nbart_swir_1.
<osgeo.gdal.Dataset; proxy of <Swig Object of type 'GDALDatasetShadow *' at 0x000001E815D5BD80> >
Combining VRTs into temp. file for band: nbart_swir_2.
<osgeo.gdal.Dataset; proxy of <Swig Object of type 'GDALDatasetShadow *' at 0x000001E815D5BD80> >
Combining VRTs into temp. file for band: oa_fmask

## Extract datetimes from vrt file for each band

In [57]:
# loop each band and extract datetimes for each
vrt_dt_dict = satfetcher.get_vrt_file_datetimes(vrt_file_dict)

Extracting datetimes for VRTs per band.
Extracting datetimes from VRTs for band: nbart_blue.
Extracting datetimes from VRTs for band: nbart_green.
Extracting datetimes from VRTs for band: nbart_red.
Extracting datetimes from VRTs for band: nbart_nir.
Extracting datetimes from VRTs for band: nbart_swir_1.
Extracting datetimes from VRTs for band: nbart_swir_2.
Extracting datetimes from VRTs for band: oa_fmask.
Extracted 7 band vrt datetimes successfully.


## Combine VRT files and datetimes into a single dict

In [58]:
# combine vrt files and datetimes for each band into final dict
vrt_dict = satfetcher.prepare_full_vrt_dicts(vrt_file_dict, vrt_dt_dict)

Combining vrt files and datetimes per band.
Combined vrt files and datetimes per band successfully.


## Build xarray datasets

In [59]:
# todo: do bounding box here

In [60]:
# create in-memory dataset from vrt dictionary
ds = satfetcher.build_xr_datasets(vrt_dict=vrt_dict)

# show ds
#ds

Building an xarray dataset from vrt files and datetimes.
Working on dataset for band: nbart_blue
Working on dataset for band: nbart_green
Working on dataset for band: nbart_red
Working on dataset for band: nbart_nir
Working on dataset for band: nbart_swir_1
Working on dataset for band: nbart_swir_2
Working on dataset for band: oa_fmask
Built an xarray dataset successfully.


## Subset size of dataset to study area bounds

In [61]:
# subset func
ds = ds.isel(x=slice(2500, 3400), y=slice(3000, 3550))

# show ds
#ds

## Test download times

### Try raw, without dask

In [ ]:
# speed testing without dask distributed
%time ds = ds.compute()

### Try raw, without dask but with threading

In [ ]:
# speed testing without dask distributed
%time ds = ds.compute(scheduler='threads')

### Try raw, without dask but with processes

In [ ]:
# speed testing without dask distributed
%time ds = ds.compute(scheduler='processes')

### Try dask, with distributed scheduler

In [ ]:
#import dask
#from dask.distributed import Client
#client = Client(processes=True)
#client

In [ ]:
# about 47 secs with processes=false, 21 secs when True
%time ds = ds.compute()

### Try dask data arrays split and futures used

In [ ]:
import concurrent.futures 

# create compute func
def compute_da(da):
    print('Computing band: {} at date: {}'.format(da.name, da['time'].values))
    return da.compute()

In [ ]:
# split ds into seperate das
da_list = []
for dt in ds['time']:
    da = ds.sel(time=dt)
    da_list.append(da)
    
# try parallel load of all bands
num_cores = 2
with concurrent.futures.ThreadPoolExecutor(num_cores) as executor:
    %time da_list = list(executor.map(compute_da, da_list))
    
ds = xr.concat(da_list, dim='time')

In [ ]:
#2 cores = Wall time: 14min 23s

### Try dask dataset split into seperate da based on band

In [ ]:
# calc oa_fmask first on its own and determine % of cloud pixels
# remove those > threshold from dask set
# compute dask det
# mask

In [ ]:
# this is th one!

In [62]:
import concurrent.futures 

# create compute func
def compute_da_full(da):
    print('Computing data array for band: {}.'.format(da.name))
    return da.compute()

In [63]:
# get list of each band seperated out
da_list = [ds[band] for band in ds]

# try parallel load of all bands
num_cores = 8
with concurrent.futures.ThreadPoolExecutor(num_cores) as executor:
    %time da_list = list(executor.map(compute_da_full, da_list))

Computing data array for band: nbart_blue.
Computing data array for band: nbart_green.
Computing data array for band: nbart_red.
Computing data array for band: nbart_nir.
Computing data array for band: nbart_swir_1.
Computing data array for band: nbart_swir_2.
Computing data array for band: oa_fmask.
Wall time: 14min 42s


In [65]:
#ds = da_list[0].to_dataset()
mask = xr.where(ds['oa_fmask'].isin([1, 4, 5]), 1, 0)

cloud_pct = (100 - (mask.sum(['x', 'y']) / mask.count(['x', 'y'])) * 100).astype('int16')

In [67]:
#100 - ((mask.sum(['x', 'y']) / mask.count(['x', 'y']))) * 100)

In [69]:
cloud_pct.compute()

<xarray.DataArray 'oa_fmask' (time: 93)>
array([35,  0,  0,  3,  0,  0,  0,  0,  0,  0,  0, 10,  0,  4,  0,  0,  0,
        0,  0, 14, 18,  0,  0,  0,  0,  0,  0,  0, 23,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        7,  0,  0, 59,  0,  0,  9,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0, 47, 21,  0, 18,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  1,  0], dtype=int16)
Coordinates:
  * time     (time) datetime64[ns] 1990-02-09T01:24:45.881044 ... 1995-12-24T...

In [ ]:
# with parallel initialised at start
# 2 cores Wall time: 7min 35s
# 4 cores Wall time: 4min 30s
# 7 cores Wall time: 4min 19s

# without parallel initialised at start
# 1 cores Wall time: 4min 22s
# 2 cores Wall time: 4min 28s
# 4 cores Wall time: 4min 41s

## Working

## Use this to auto gen vrt to test

In [2]:
# really good test env
vrt_1_urls = [
    '/vsicurl/https://data.dea.ga.gov.au/baseline/ga_ls5t_ard_3/112/076/1990/02/09/ga_ls5t_nbart_3-0-0_112076_1990-02-09_final_band01.tif',
    '/vsicurl/https://data.dea.ga.gov.au/baseline/ga_ls5t_ard_3/112/076/1990/02/09/ga_ls5t_nbart_3-0-0_112076_1990-02-09_final_band02.tif',
    '/vsicurl/https://data.dea.ga.gov.au/baseline/ga_ls5t_ard_3/112/076/1990/02/09/ga_ls5t_nbart_3-0-0_112076_1990-02-09_final_band03.tif']
vrt1 = gdal.BuildVRT('D:/vrt_1.vrt', vrt_1_urls, separate=True).FlushCache()

vrt_2_urls = [
    '/vsicurl/https://data.dea.ga.gov.au/baseline/ga_ls5t_ard_3/112/076/1990/03/13/ga_ls5t_nbart_3-0-0_112076_1990-03-13_final_band01.tif',
    '/vsicurl/https://data.dea.ga.gov.au/baseline/ga_ls5t_ard_3/112/076/1990/03/13/ga_ls5t_nbart_3-0-0_112076_1990-03-13_final_band02.tif',
    '/vsicurl/https://data.dea.ga.gov.au/baseline/ga_ls5t_ard_3/112/076/1990/03/13/ga_ls5t_nbart_3-0-0_112076_1990-03-13_final_band03.tif']
vrt2 = gdal.BuildVRT('D:/vrt_2.vrt', vrt_2_urls, separate=True).FlushCache()

vrt_3_urls = [
    '/vsicurl/https://data.dea.ga.gov.au/baseline/ga_ls5t_ard_3/111/076/1990/03/22/ga_ls5t_nbart_3-0-0_111076_1990-03-22_final_band01.tif',
    '/vsicurl/https://data.dea.ga.gov.au/baseline/ga_ls5t_ard_3/111/076/1990/03/22/ga_ls5t_nbart_3-0-0_111076_1990-03-22_final_band02.tif',
    '/vsicurl/https://data.dea.ga.gov.au/baseline/ga_ls5t_ard_3/111/076/1990/03/22/ga_ls5t_nbart_3-0-0_111076_1990-03-22_final_band03.tif']
vrt3 = gdal.BuildVRT('D:/vrt_3.vrt', vrt_3_urls, separate=True).FlushCache()


vrt_4_urls = [
    '/vsicurl/https://data.dea.ga.gov.au/baseline/ga_ls5t_ard_3/112/076/1990/03/29/ga_ls5t_nbart_3-0-0_112076_1990-03-29_final_band01.tif',
    '/vsicurl/https://data.dea.ga.gov.au/baseline/ga_ls5t_ard_3/112/076/1990/03/29/ga_ls5t_nbart_3-0-0_112076_1990-03-29_final_band02.tif',
    '/vsicurl/https://data.dea.ga.gov.au/baseline/ga_ls5t_ard_3/112/076/1990/03/29/ga_ls5t_nbart_3-0-0_112076_1990-03-29_final_band03.tif']
vrt4 = gdal.BuildVRT('vrt_4.vrt', vrt_4_urls, separate=True).FlushCache()

# add to list
vrt_list = ['vrt_1.vrt', 'vrt_2.vrt', 'vrt_3.vrt', 'vrt_4.vrt']
vrt_out = gdal.BuildVRT('D:/vrt_all.vrt', vrt_list, separate=True, bandList=[1]).FlushCache()


# read it in to memory and decode it
#vrt_all = tmp.read().decode("utf-8")

# setup chunks
#chunks = {'band': 1, 'x': 'auto', 'y': 'auto'}
#ds = xr.open_rasterio('vrt_all.vrt', chunks=chunks)
#ds = ds.isel(x=slice(2500, 3000), y=slice(2500, 3000))
#ds.compute()

In [3]:
with rasterio.Env(CURL_CA_BUNDLE=certifi.where()) as env:
    out = rasterio.open('D:/vrt_all.vrt')

In [7]:
xr.open_rasterio('D:/vrt_all.vrt').compute()

<xarray.DataArray (band: 2, y: 7071, x: 7951)>
array([[[-999, -999, -999, ..., -999, -999, -999],
        [-999, -999, -999, ..., -999, -999, -999],
        [-999, -999, -999, ..., -999, -999, -999],
        ...,
        [-999, -999, -999, ..., -999, -999, -999],
        [-999, -999, -999, ..., -999, -999, -999],
        [-999, -999, -999, ..., -999, -999, -999]],

       [[-999, -999, -999, ..., -999, -999, -999],
        [-999, -999, -999, ..., -999, -999, -999],
        [-999, -999, -999, ..., -999, -999, -999],
        ...,
        [-999, -999, -999, ..., -999, -999, -999],
        [-999, -999, -999, ..., -999, -999, -999],
        [-999, -999, -999, ..., -999, -999, -999]]], dtype=int16)
Coordinates:
  * band     (band) int32 1 2
  * y        (y) float64 -2.451e+06 -2.451e+06 ... -2.663e+06 -2.663e+06
  * x        (x) float64 5.634e+05 5.634e+05 5.635e+05 ... 8.019e+05 8.019e+05
Attributes:
    transform:   (30.0, 0.0, 563385.0, 0.0, -30.0, -2450985.0)
    crs:         +init=epsg:32650
    res:         (30.0, 30.0)
    is_tiled:    1
    nodatavals:  (-999.0, -999.0)
    scales:      (1.0, 1.0)
    offsets:     (0.0, 0.0)